In [1]:
import numpy as np
import xarray as xr
import intake

In [2]:
from pyrte_rrtmgp import rrtmgp_cloud_optics, rrtmgp_gas_optics
from pyrte_rrtmgp.data_types import (
    CloudOpticsFiles,
    GasOpticsFiles,
    OpticsProblemTypes,
)
from pyrte_rrtmgp.rte_solver import rte_solve

In [3]:
import warnings

# Suppress specific FutureWarnings matching the message pattern when using cat[...].to_dask()
warnings.filterwarnings(
    "ignore",
    message=".*The return type of `Dataset.dims` will be changed.*",
    category=FutureWarning,
)

# Read data

Zoom level 5 is 12288 points (roughly 5 degrees); each zoom level (max 11, min 0) is 4x more, fewer points or 2x higher in grid density

Perhaps chunks should be introduced at this stage? 

In [4]:
cat = intake.open_catalog('https://digital-earths-global-hackathon.github.io/catalog/catalog.yaml')['online']

In [5]:
zoom = 5
data = cat["icon_d3hp003feb"](zoom=zoom).to_dask()

In [6]:
# Local copy
# data = xr.open_dataset("/Users/robert/Codes/hk25/data/PT15M_inst_z5_atm", consolidated=True, engine='zarr')

In [15]:
data

<xarray.Dataset> Size: 2GB
Dimensions:     (crs: 1, time: 96, pressure: 30, cell: 12288, pressure_h: 31)
Coordinates:
  * crs         (crs) float32 4B nan
  * pressure    (pressure) int64 240B 5 10 20 50 ... 92500 95000 97500 100000
  * time        (time) datetime64[ns] 768B 2020-02-01T00:15:00 ... 2020-02-02
  * pressure_h  (pressure_h) float64 248B 1.0 7.071 ... 9.874e+04 1.005e+05
  * cell        (cell) float32 49kB 0.0 1.0 2.0 ... 1.229e+04 1.229e+04
Data variables: (12/28)
    hur         (time, pressure, cell) float32 142MB ...
    hus         (time, pressure, cell) float32 142MB 4.426e-06 ... 0.01778
    orog        (cell) float32 49kB ...
    pr          (time, cell) float32 5MB ...
    psl         (time, cell) float32 5MB ...
    qall        (time, pressure, cell) float32 142MB ...
    ...          ...
    ch4         float64 8B 1.65e-06
    n2o         float64 8B 3.06e-07
    n2          float64 8B 0.7808
    o2          float64 8B 0.2095
    co          float64 8B 0.0
    p2          (time, pressure, cell) int64 283MB 5 5 5 ... 100000 100000

# Transform the data to the form needed to compute fluxes 

## Pressure values on levels
 Top level pressure is arbitrary, bottom level presure would normally be surface pressure but these data have been 
 interpolated to fixed pressures so the surface pressure can be well below 1000 hPa 

In [8]:
data["pressure_h"] = xr.concat([
                        xr.DataArray([1], dims="pressure_h"),
                        xr.DataArray(((data.pressure.values[1:] * data.pressure.values[:-1]) ** 0.5), dims="pressure_h"),
                        xr.DataArray([100500], dims="pressure_h")
                    ], dim = "pressure_h")

## Temperature on levels
Linear interpolation of temperature in pressure
  Temperature at top level is same as top layer 
  Temperature at bottom level is surface T (might be colder than lowest layer when elevation is high) 


In [9]:
data["ta_h"] = data.ta.interp(pressure=data.pressure_h)
data["ta_h"][:,  0, :] = data.ta[:,  0, :]
data["ta_h"][:, -1, :] = data.ts

## Water vapor - convert specific humidity to molar mixing ratio 

Molar mixing ratio assuming specific humidity is water mass/dry air mass (not quite correct)
  Md, Mw are molar mases of dry air and water vapor 


In [10]:
Md = 0.0289652
Mw = 0.018016
data["h2o"] = data.hus * (Md/Mw)

## Ozone from monthly-mean ERA5 interpolated onto HEALPix grid at zoom levels 8 and below

In [11]:
if zoom <= 8:
    data["o3"] = cat["ERA5"](zoom=zoom)\
        .to_dask()\
        .sel(time="2020-02-01", method="nearest")\
        .o3.interp(level=data.pressure)\
        .reset_coords(("lat", "lon", "level", "time"))\
        .drop_vars(("lat", "lon", "level", "time"))\
        .o3

# This is actually a mass fraction; need to set to vmr 
# also need to change/delete units
    
# For zoom > 8 we need to interpolate in space too - probably nearest neighbor 

## Well-mixed greenhouse gases (these are pre-industrial values, should update) 

In [12]:
gas_values = {
        "co2": 348e-6,
        "ch4": 1650e-9,
        "n2o": 306e-9,
        "n2": 0.7808,
        "o2": 0.2095,
        "co": 0.0,
}
for gas_name, value in gas_values.items():
    data[gas_name] = value

## Cloud properties 
  Data set includes only `qall` all hydrometeors 
  Assume all clouds > 263 are liquid, everything else is ice (could refine)
  Convert from MMR to vertically-integrated LWP, IWP 

In [ ]:
data["lwp"] = xr.where(data.ta >= 263., data.qall, 0)  
data["iwp"] = xr.where(data.ta <  263., data.qall, 0)  

# Liquid and ice effective sizes in microns 
data["rel"] = xr.where(data.lwp > 0., 10., 0)  
data["rei"] = xr.where(data.iwp > 0,  35., 0)  

# RTE and RRTMPG initialization 

In [13]:
cloud_optics_lw = rrtmgp_cloud_optics.load_cloud_optics(
    cloud_optics_file=CloudOpticsFiles.LW_BND
)
gas_optics_lw = rrtmgp_gas_optics.load_gas_optics(
    gas_optics_file=GasOpticsFiles.LW_G128
)

cloud_optics_sw = rrtmgp_cloud_optics.load_cloud_optics(
    cloud_optics_file=CloudOpticsFiles.SW_BND
)
gas_optics_sw = rrtmgp_gas_optics.load_gas_optics(
    gas_optics_file=GasOpticsFiles.SW_G112
)

In [14]:
# Will the gas optics work? 
data["p2"] = data["pressure"].broadcast_like(data.ta)

var_mapping = {"p2":"pres_layer", 
               "pressure_h":"pres_level", 
               "ta":"temp_layer", 
               "ta_h":"temp_level"}

gas_optics_sw.compute_gas_optics(
                data.rename_dims({"pressure":"layer", 
                                  "pressure_h":"level"})\
                    .rename(var_mapping).isel(time=6),
                problem_type=OpticsProblemTypes.TWO_STREAM, 
                add_to_input=False,
            )

ValueError: could not convert string to float: 'kg kg**-1'

# Compute fluxes - shortwave 

In [ ]:
fluxes = rte_solve(
    xr.merge(
        [cloud_optics_lw.compute_cloud_optics(data).add_to(
            gas_optics_lw.compute_gas_optics(
                data, 
                problem_type=OpticsProblemTypes.ABSORPTION, 
                add_to_input=False,
            ), 
        ), 
        xr.Dataset(data_vars = {"surface_emissivity":0.98})],
    ).rename_dims({"pressure":"layer", 
                   "pressure_h":"level"}), 
    add_to_input = False,
)

In [ ]:
o3 = cat["ERA5"](zoom=zoom)\
        .to_dask()\
        .sel(time="2020-02-01", method="nearest").o3.interp(level=data.pressure)

# 
# How to combine with existing data? 